In [10]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
from slugify import slugify

warnings.simplefilter("ignore")

In [2]:
def get_most_import_players(line_type):
    server = 'localhost\SQLEXPRESS'
    database = 'nba_game_data'

    sql = f'''
    with minutesPerGame as (
    SELECT [SEASON_YEAR]
          ,[PLAYER_ID]
          ,avg(MIN) avg_min
      FROM [nba_game_data].[dbo].[PlayerGameLogs]
      where yearSeason =2024
      group by [SEASON_YEAR]
          ,[PLAYER_ID]
    )

    SELECT pgl.[SEASON_YEAR]
          ,pgl.[PLAYER_ID]
          ,pgl.[PLAYER_NAME]
          ,pgl.[NICKNAME]
          ,pgl.[TEAM_ID]
          ,pgl.[TEAM_ABBREVIATION]
          ,pgl.[TEAM_NAME]
          ,pgl.[GAME_ID]
          ,pgl.[GAME_DATE]

          ,{line_type} as PTS
      FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl
       left outer join minutesPerGame mpg
      on 
      pgl.[PLAYER_ID] = mpg.[PLAYER_ID]
      and pgl.[SEASON_YEAR] = mpg.[SEASON_YEAR]

      where yearSeason =2024
      and  mpg.avg_min > 20
    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    df = pd.read_sql(sql,cnxn)
    
    sql = f'''
    SELECT *
      FROM PlayersCurrentTeam
    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    players_current_team = pd.read_sql(sql,cnxn)

    average_pts = df.groupby('PLAYER_NAME')['PTS'].mean().reset_index()

    # Rename the columns for clarity
    average_pts.rename(columns={'PTS': 'AVG_PTS'}, inplace=True)
    
    df_with_avg = pd.merge(df, average_pts, on='PLAYER_NAME')
    
    teammates = df_with_avg.groupby(['TEAM_ID', 'GAME_ID'])['PLAYER_ID'].apply(list).reset_index()
    teammates = teammates.rename(columns={'PLAYER_ID':'Players_Who_Played_In_That_Game'})
    teammate_joins = df_with_avg.merge(teammates, on=['TEAM_ID','GAME_ID'])
    
#     teammate_joins['All_Teammates'] = teammate_joins['PLAYER_ID'].apply(lambda x: list(teammates_dict[x]) if x in teammates_dict else [])
    
    teammate_joins['Players_Who_Played_In_That_Game'] = teammate_joins['Players_Who_Played_In_That_Game'].apply(
        lambda x: literal_eval(x) if isinstance(x, str) else x
    )

    # Proceed with creating the set of all teammates
    teammates_dict = {}
    
    teammate_joins['All_Teammates'] = teammate_joins['PLAYER_ID'].apply(lambda x: list(teammates_dict[x]) if x in teammates_dict else [])
    
    for index, row in teammate_joins.iterrows():
        player_id = row['PLAYER_ID']
        teammates = set(row['Players_Who_Played_In_That_Game']) - {player_id}  # Remove the player's own ID
        if player_id not in teammates_dict:
            teammates_dict[player_id] = teammates
        else:
            teammates_dict[player_id].update(teammates)

    # Map the teammates set back to the DataFrame, creating a new column 'All_Teammates'
    teammate_joins['All_Teammates'] = teammate_joins['PLAYER_ID'].apply(lambda x: list(teammates_dict[x]))

    # Ensure no duplication within 'All_Teammates'
    teammate_joins['All_Teammates'] = teammate_joins['All_Teammates'].apply(lambda x: list(set(x)))
    
    player_to_team_map = players_current_team.set_index('PLAYER_ID')['TEAM_ABBREVIATION'].to_dict()

    # Placeholder DataFrame for importance scores and games played
    importance_scores_df = pd.DataFrame(columns=['PLAYER_ID', 'TEAMMATE_ID','RAW_IMPORTANCE_SCORE', 'IMPORTANCE_SCORE', 'GAMES_PLAYED','TOTAL_GAMES_PLAYED'])

    for player_id in teammate_joins['PLAYER_ID'].unique():
        player_games = teammate_joins[teammate_joins['PLAYER_ID'] == player_id]

        number_of_games = len(player_games)

        current_team_id = player_to_team_map.get(player_id)
        all_teammates = set()

        for index, row in player_games.iterrows():
            all_teammates.update(row['All_Teammates'])

        for teammate_id in all_teammates:
            teammate_current_team_id = player_to_team_map.get(teammate_id)
            if teammate_current_team_id != current_team_id:
                continue

            pts_with_teammate = []
            pts_without_teammate = []
            games_with_teammate = 0

            for index, game in player_games.iterrows():
                if teammate_id in game['Players_Who_Played_In_That_Game']:
                    pts_with_teammate.append(game['PTS'] - game['AVG_PTS'])
                    games_with_teammate += 1
                else:
                    pts_without_teammate.append(game['PTS'] - game['AVG_PTS'])

            avg_diff_with = sum(pts_with_teammate) / len(pts_with_teammate) if pts_with_teammate else 0
            avg_diff_without = sum(pts_without_teammate) / len(pts_without_teammate) if pts_without_teammate else 0

            importance_score = avg_diff_with - avg_diff_without

            temp = pd.DataFrame([{
                'PLAYER_ID': player_id,
                'TEAMMATE_ID': teammate_id,
                'RAW_IMPORTANCE_SCORE':importance_score,
                'IMPORTANCE_SCORE': abs(importance_score),
                'GAMES_PLAYED': games_with_teammate,
                'TOTAL_GAMES_PLAYED': number_of_games
            }])

            importance_scores_df = pd.concat([importance_scores_df, temp], ignore_index=True)



    importance_scores_df['PCT_OF_GAMES_PLAYED_TOGETHER'] = importance_scores_df['GAMES_PLAYED']/importance_scores_df['TOTAL_GAMES_PLAYED']
    importance_scores_df = importance_scores_df.loc[importance_scores_df['PCT_OF_GAMES_PLAYED_TOGETHER']>=.25].reset_index(drop=True)
    
    importance_scores_df = importance_scores_df.sort_values(by=['PLAYER_ID', 'IMPORTANCE_SCORE'], ascending=[True, False])
    importance_scores_df = importance_scores_df.drop_duplicates(subset=['PLAYER_ID'], keep='first')
    
    importance_scores_df = importance_scores_df[['PLAYER_ID','TEAMMATE_ID','RAW_IMPORTANCE_SCORE']]
    return importance_scores_df.rename(columns={'TEAMMATE_ID' : f'{line_type}_TEAMMATE_ID','RAW_IMPORTANCE_SCORE':f'{line_type}_IMPORTANCE_SCORE'})

In [3]:
tracker = 0 
lines = ['PTS','REB','AST','STL','BLK','TOV','FG3M','DD2']
for i in lines:
    if tracker == 0:
        output =get_most_import_players(i)
        tracker +=1
    else:
        output = pd.merge(output,get_most_import_players(i),on='PLAYER_ID')
output

,PLAYER_ID,PTS_TEAMMATE_ID,PTS_IMPORTANCE_SCORE,REB_TEAMMATE_ID,REB_IMPORTANCE_SCORE,AST_TEAMMATE_ID,AST_IMPORTANCE_SCORE,STL_TEAMMATE_ID,STL_IMPORTANCE_SCORE,BLK_TEAMMATE_ID,BLK_IMPORTANCE_SCORE,TOV_TEAMMATE_ID,TOV_IMPORTANCE_SCORE,FG3M_TEAMMATE_ID,FG3M_IMPORTANCE_SCORE,DD2_TEAMMATE_ID,DD2_IMPORTANCE_SCORE
0,2544,203076,4.775362,1627752,-1.601449,1627752,2.224638,1629629,0.691228,1629060,-0.371429,203076,0.615942,1627752,-0.934783,1629020,0.176667
1,101108,201939,-6.466667,202691,-2.290323,203952,-3.533333,203952,-0.933333,201939,-0.433333,201939,-1.400000,201939,-1.766667,203952,0.133333
2,200768,1631170,6.378788,1629130,-2.128571,1629130,1.028571,1629130,-1.000000,1629130,0.428571,202710,-1.210000,1631170,1.878788,1628997,-0.076923
3,201142,203994,9.088889,1628960,1.733333,203994,1.066667,203994,0.644444,203994,0.666667,203994,-3.666667,203994,0.977778,203994,0.244444
4,201143,204001,-3.168269,201950,-2.237805,1630573,2.931818,1630573,0.568182,1628401,0.738095,1630573,-0.272727,1630573,-1.386364,1627759,-0.201220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,1641722,1630700,9.765957,1630530,-2.198261,1627742,-1.340909,1627742,-1.250000,203468,0.117647,1630529,-0.948837,1630700,2.000000,1630529,0.000000
222,1641726,1629029,5.617647,203501,-4.921569,1627884,0.893939,1627884,-0.424242,203501,-2.039216,203501,-1.666667,1629029,0.000000,203501,-0.490196
223,1641731,1628398,5.549020,1628398,2.176471,1630557,-1.288462,1630557,0.865385,1628398,0.803922,1628398,-1.049020,1628398,1.039216,1629673,0.019608
224,1641739,1629680,3.560000,1629028,-2.708333,1629680,-0.540000,1629680,0.540000,1631133,-0.604167,1631133,-0.895833,1629680,0.780000,1629028,-0.062500


In [30]:
if len(output) > 0:
    # Presuming 'server' and 'database' variables are defined elsewhere
    cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
    cursor = cnxn.cursor()
    
    cursor.execute("DELETE FROM InfluentialPlayersV1")
    cnxn.commit()

    # SQL INSERT statement for the 'InfluentialPlayersV1' table
    insert_stmt = '''
    INSERT INTO InfluentialPlayersV1 (
        PLAYER_ID, PTS_TEAMMATE_ID, PTS_IMPORTANCE_SCORE, 
        REB_TEAMMATE_ID, REB_IMPORTANCE_SCORE, 
        AST_TEAMMATE_ID, AST_IMPORTANCE_SCORE, 
        STL_TEAMMATE_ID, STL_IMPORTANCE_SCORE, 
        BLK_TEAMMATE_ID, BLK_IMPORTANCE_SCORE, 
        TOV_TEAMMATE_ID, TOV_IMPORTANCE_SCORE, 
        FG3M_TEAMMATE_ID, FG3M_IMPORTANCE_SCORE, 
        DD2_TEAMMATE_ID, DD2_IMPORTANCE_SCORE
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in output.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Make sure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
                row['PLAYER_ID'], row['PTS_TEAMMATE_ID'], row['PTS_IMPORTANCE_SCORE'], 
                row['REB_TEAMMATE_ID'], row['REB_IMPORTANCE_SCORE'], 
                row['AST_TEAMMATE_ID'], row['AST_IMPORTANCE_SCORE'], 
                row['STL_TEAMMATE_ID'], row['STL_IMPORTANCE_SCORE'], 
                row['BLK_TEAMMATE_ID'], row['BLK_IMPORTANCE_SCORE'], 
                row['TOV_TEAMMATE_ID'], row['TOV_IMPORTANCE_SCORE'], 
                row['FG3M_TEAMMATE_ID'], row['FG3M_IMPORTANCE_SCORE'], 
                row['DD2_TEAMMATE_ID'], row['DD2_IMPORTANCE_SCORE']
            )        
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # You can choose to either break or continue based on your preference

    cursor.close()
    cnxn.close()


In [29]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame
# Replace with your actual DataFrame
# df = ...

import pandas as pd

def pandas_to_sql_type(pandas_type):
    if pd.api.types.is_string_dtype(pandas_type):
        return 'NVARCHAR(MAX)'
    elif pd.api.types.is_integer_dtype(pandas_type):
        return 'INT'
    elif pd.api.types.is_float_dtype(pandas_type):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(pandas_type):
        return 'BIT'
    elif pd.api.types.is_datetime64_dtype(pandas_type):
        return 'DATETIME'
    else:
        # Handle other data types as needed
        return 'NVARCHAR(MAX)'  # Default to NVARCHAR(MAX) for unsupported types

# Example usage:
# sql_type = pandas_to_sql_type(df[column].dtype)


def create_table_statement(df, table_name):
    columns = df.columns
    sql_columns = []
    for column in columns:
        sql_type = pandas_to_sql_type(df[column].dtype)
        sql_columns.append(f"[{column}] {sql_type}")

    columns_str = ",\n  ".join(sql_columns)
    sql_statement = f"CREATE TABLE {table_name} (\n  {columns_str}\n);"
    return sql_statement

# Replace 'YourTableName' with your desired table name
# table_name = 'gameStatsTrackingkV3'
sql_statement = create_table_statement(output, 'InfluentialPlayersV1')
print(sql_statement)

CREATE TABLE InfluentialPlayersV1 (
  [PLAYER_ID] NVARCHAR(MAX),
  [PTS_TEAMMATE_ID] NVARCHAR(MAX),
  [PTS_IMPORTANCE_SCORE] FLOAT,
  [REB_TEAMMATE_ID] NVARCHAR(MAX),
  [REB_IMPORTANCE_SCORE] FLOAT,
  [AST_TEAMMATE_ID] NVARCHAR(MAX),
  [AST_IMPORTANCE_SCORE] FLOAT,
  [STL_TEAMMATE_ID] NVARCHAR(MAX),
  [STL_IMPORTANCE_SCORE] FLOAT,
  [BLK_TEAMMATE_ID] NVARCHAR(MAX),
  [BLK_IMPORTANCE_SCORE] FLOAT,
  [TOV_TEAMMATE_ID] NVARCHAR(MAX),
  [TOV_IMPORTANCE_SCORE] FLOAT,
  [FG3M_TEAMMATE_ID] NVARCHAR(MAX),
  [FG3M_IMPORTANCE_SCORE] FLOAT,
  [DD2_TEAMMATE_ID] NVARCHAR(MAX),
  [DD2_IMPORTANCE_SCORE] FLOAT
);


In [4]:
output.loc[output['PLAYER_ID']==1630178]

,PLAYER_ID,PTS_TEAMMATE_ID,PTS_IMPORTANCE_SCORE,REB_TEAMMATE_ID,REB_IMPORTANCE_SCORE,AST_TEAMMATE_ID,AST_IMPORTANCE_SCORE,STL_TEAMMATE_ID,STL_IMPORTANCE_SCORE,BLK_TEAMMATE_ID,BLK_IMPORTANCE_SCORE,TOV_TEAMMATE_ID,TOV_IMPORTANCE_SCORE,FG3M_TEAMMATE_ID,FG3M_IMPORTANCE_SCORE,DD2_TEAMMATE_ID,DD2_IMPORTANCE_SCORE
165,1630178,1626162,-2.035885,201587,-0.839655,203954,1.193182,201587,0.691379,1626162,-0.279904,203954,-0.856061,202699,1.381818,203954,0.181818


In [5]:
all_ids = pd.concat([
    output['PTS_TEAMMATE_ID'],
    output['REB_TEAMMATE_ID'],
    output['AST_TEAMMATE_ID'],
    output['STL_TEAMMATE_ID'],
    output['BLK_TEAMMATE_ID'],
    output['TOV_TEAMMATE_ID'],
    output['FG3M_TEAMMATE_ID'],
    output['DD2_TEAMMATE_ID']
])

# Get unique values from this series
distinct_ids = all_ids.unique().tolist()

In [6]:
import_players = pd.DataFrame(distinct_ids,columns = ['PlayerID'])
import_players

,PlayerID
0,203076
1,201939
2,1631170
3,203994
4,204001
...,...
199,1629638
200,203084
201,1631110
202,1628969


In [7]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

sql = f'''

SELECT distinct
      pgl.[PLAYER_ID]
      ,pgl.[PLAYER_NAME]

  FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl


  where yearSeason =2024
'''
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
all_players = pd.read_sql(sql,cnxn)

In [11]:
all_players['PlayerSlug'] = all_players['PLAYER_NAME'].apply(lambda x: slugify(x))
all_players

,PLAYER_ID,PLAYER_NAME,PlayerSlug
0,2544,LeBron James,lebron-james
1,101108,Chris Paul,chris-paul
2,200768,Kyle Lowry,kyle-lowry
3,200782,P.J. Tucker,p-j-tucker
4,201142,Kevin Durant,kevin-durant
...,...,...,...
535,1641871,Duop Reath,duop-reath
536,1641877,Nathan Mensah,nathan-mensah
537,1641926,Dexter Dennis,dexter-dennis
538,1641931,Onuralp Bitim,onuralp-bitim


In [17]:
import_players = pd.merge(import_players,all_players, left_on ='PlayerID',right_on = 'PLAYER_ID', how='left' )

In [12]:
injuries = pd.read_csv('nba-injury-report.csv')

In [13]:
injuries['PlayerSlug'] = injuries['Player'].apply(lambda x: slugify(x))

In [19]:
import_players = pd.merge(import_players,injuries, on ='PlayerSlug', how='left' )

In [27]:
import_players.loc[import_players['Status']=='Game Time Decision']

,PlayerID,PLAYER_ID,PLAYER_NAME,PlayerSlug,Player,Team,Pos,Injury,Status,Est. Return
4,204001,204001,Kristaps Porzingis,kristaps-porzingis,Kristaps Porzingis,BOS,F,Ankle,Game Time Decision,Subscribers Only
18,202711,202711,Bojan Bogdanovic,bojan-bogdanovic,Bojan Bogdanovic,NYK,F,Calf,Game Time Decision,Subscribers Only
20,201587,201587,Nicolas Batum,nicolas-batum,Nicolas Batum,PHI,F,Hamstring,Game Time Decision,Subscribers Only
24,2544,2544,LeBron James,lebron-james,LeBron James,LAL,F,Ankle,Game Time Decision,Subscribers Only
31,1630172,1630172,Patrick Williams,patrick-williams,Patrick Williams,CHI,F,Foot,Game Time Decision,Subscribers Only
33,1630591,1630591,Jalen Suggs,jalen-suggs,Jalen Suggs,ORL,G,Groin,Game Time Decision,Subscribers Only
40,1630168,1630168,Onyeka Okongwu,onyeka-okongwu,Onyeka Okongwu,ATL,C,Toe,Game Time Decision,Subscribers Only
42,203484,203484,Kentavious Caldwell-Pope,kentavious-caldwell-pope,Kentavious Caldwell-Pope,DEN,G,Hamstring,Game Time Decision,Subscribers Only
43,201143,201143,Al Horford,al-horford,Al Horford,BOS,C,Toe,Game Time Decision,Subscribers Only
47,203078,203078,Bradley Beal,bradley-beal,Bradley Beal,PHX,G,Hamstring,Game Time Decision,Subscribers Only


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import csv
from time import sleep

# Set up the Selenium driver (e.g., Chrome, Firefox)
driver = webdriver.Chrome()

try:
    # Go to the webpage
    driver.get("https://www.rotowire.com/basketball/injury-report.php")

    # Wait for the data to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "webix_ss_body")))
    sleep(5)
    print(driver.page_source)
    # Now that the page is loaded, you can start scraping the data
    # Get the container that holds all the rows
    container = driver.find_element(By.CLASS_NAME, "webix_ss_center_scroll")

    # Get all the player rows within the container
    player_rows = container.find_elements(By.XPATH, ".//div[contains(@class, 'webix_row_select')]")
    print(player_rows)
    # Initialize an empty list to hold all player data
    data = []

    # Loop through each player row and extract the data
    for row in player_rows:
        # Get all cells within the row
        cells = row.find_elements(By.CLASS_NAME, "webix_cell")
        if len(cells) >= 6:  # Ensure there are enough cells for player data
            player_data = {
                'Player': cells[0].text.strip(),
                'Team': cells[1].text.strip(),
                'Pos': cells[2].text.strip(),
                'Injury': cells[3].text.strip(),
                'Status': cells[4].text.strip(),
                'Est. Return': cells[5].text.strip() if len(cells) > 5 else ''
            }
            data.append(player_data)


#     # Write the data to a CSV file
#     with open('/mnt/data/injuries.csv', 'w', newline='') as file:
#         fieldnames = ['Player', 'Team', 'Pos', 'Injury', 'Status', 'Est. Return']
#         writer = csv.DictWriter(file, fieldnames=fieldnames)
#         writer.writeheader()
#         for entry in data:
#             writer.writerow(entry)
finally:
    # Close the browser when done
    driver.quit()

# Output the path to the CSV file
# print("/mnt/data/injuries.csv")
data